# [Chialisp Modules](https://chialisp.com/operators)
`mod`, `include`, `defun`, `defun-inline`, `lambda`, `defmacro`, `defconstant`

## `defun` vs `defmacro`
[What is the difference between defn and defmacro?](https://stackoverflow.com/questions/3667403/what-is-the-difference-between-defn-and-defmacro)

> The difference between functions and macros is that on a function call first the arguments of the function are evaluated then the body of the function is evaluated using the arguments.

> Macros on the other hand describe a transformation from one piece of code to another. Any evaluation takes place after the transformation.

> This means that arguments may be evaluated multiple times or not at all. As an example or is a macro. If the first argument of or is false, the second argument will never be evaluated. If or were a function, this would not be possible, because the arguments would always be evaluated before the function runs.

### [Macros](https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros)
> You can also define macros within a module, which act as inline functions. When a previously defined macro operator is encountered, it "rewrites" the existing statement using the macro, passing along the arguments as literals (ie. they are not evaluated).

- [`qq` and `unquote`](../quote/README.md)
- [defmacro, qq, unquote](https://chialisp.com/examples#defmacro)
- [macros.clvm](https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm)
- [utility_macros.clib](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib)


## Chialisp Macro
- Similar to functions, but they operate at a higher level of abstraction, allowing them to manipulate the code itself rather than just the data.
- Can perform complex transformation not just text replacing.
- Define using regular Chialisp syntax and can be called like function

- Similar to C/C++ macro as it's expanded during compiled time?
- Macro may or maynot faster than function.


In [5]:
clsp = """
(mod (fn . lst)
    (defun map (fn lst)
        (if lst
            (c
                (a fn (f lst))
                (map fn (r lst))
            )
            () 
        )
    )
    (map fn lst)
)
"""
map = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(map)

# square the items in the list
fn = Program.to(
    compile_clvm_text("(lambda v (* v v))", search_paths=["."])
)
solution = Program.to([fn] + list(range(1, 4)))
print("[bold bright_green]solution:")
print_program(solution)

cost, result = map.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(f"[bold green reverse]{cost}")
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod (fn . lst)                                                                                               
   3     (defun map (fn lst)                                                                                       
   4         (if lst                                                                                               
   5             (c                                                                                                
   6                 (a fn (f lst))                                                                                
   7                 (map fn (r lst))                                                                              
   8             )                                                                                                 
   9             ()                                                                                                
  10         )                                                                                                     
  11     )                                                                                                         
  12     (map fn lst)                                                                                              
  13 )                                                                                                             
  14                                                                                                               

clvm:

(a (q 2 2 (c 2 (c 5 (c 7 ())))) (c (q 2 (i 11 (q 4 (a 5 19) (a 2 (c 2 (c 5 (c 27 ()))))) ()) 1) 1))                

solution:

((* 1 1) 1 2 3)                                                                                                    

cost:

7428

result:

(q 4 9)                                                                                                            

In [ ]:
fn = Program.to(
    compile_clvm_text("(lambda v (* v v))", search_paths=["."])
)
"""
(c (a (lambda v (* v v)) 2)
"""

In [1]:
clsp = """
(mod (a b c d)
    (defmacro map-macro items
        (if (r items)
            (qq
                (c 
                    (+ (unquote (f items)) (unquote (f items)))
                    (unquote (c map-macro (r items)))
                )
            )
            (qq (list (+ (unquote (f items)) (unquote (f items)))))
        )
    )
    (map-macro a b c d)
)
"""
map = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(map)

# square the items in the list
#fn = Program.to(
#    compile_clvm_text("(lambda v (* v v))", search_paths=["."])
#)
#solution = Program.to([fn] + list(range(1, 11)))
solution = Program.to([1, 2, 3, 4])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = map.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(f"[bold green reverse]{cost}")
print("[bold bright_green]result:")
print_program(result)
result = map.run(solution)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod (a b c d)                                                                                                
   3     (defmacro map-macro items                                                                                 
   4         (if (r items)                                                                                         
   5             (qq                                                                                               
   6                 (c                                                                                            
   7                     (+ (unquote (f items)) (unquote (f items)))                                               
   8                     (unquote (c map-macro (r items)))                                                         
   9                 )                                                                                             
  10             )                                                                                                 
  11             (qq (list (+ (unquote (f items)) (unquote (f items)))))                                           
  12         )                                                                                                     
  13     )                                                                                                         
  14     (map-macro a b c d)                                                                                       
  15 )                                                                                                             
  16                                                                                                               

clvm:

(c (+ 2 2) (c (+ 5 5) (c (+ 11 11) (c (+ 23 23) ()))))                                                             

solution:

...

3704

result:

(a 4 6 8)                                                                                                          

result:

(a 4 6 8)                                                                                                          

In [1]:
clsp = Path('macros.clib').read_text()
print_clsp(clsp)

   1 (                                                                                                             
   2     ; "assert" is a macro that wraps repeated instances of "if"                                               
   3     ; usage: (assert A0 A1 ... An R)                                                                          
   4     ; all of A0, A1, ... An must evaluate to non-null, or an exception is raised                              
   5     ; return the value of R (if we get that far)                                                              
   6     (defmacro assert items                                                                                    
   7         (if (r items)                                                                                         
   8             (list if (f items) (c assert (r items)) (q . (x)))                                                
   9             (f items)                                                                                         
  10         )                                                                                                     
  11     )                                                                                                         
  12                                                                                                               
  13     ;; boolean and macro                                                                                      
  14     ;; This lets you write something like (if (and COND1 COND2 COND3) (do-something) (do-something-else))     
  15     (defmacro and ARGS                                                                                        
  16         (if ARGS                                                                                              
  17             (qq                                                                                               
  18                 (if (unquote (f ARGS))                                                                        
  19                     (unquote (c and (r ARGS)))                                                                
  20                     ()                                                                                        
  21                 )                                                                                             
  22             )                                                                                                 
  23             1                                                                                                 
  24         )                                                                                                     
  25     )                                                                                                         
  26                                                                                                               
  27     ;; boolean or macro                                                                                       
  28     ;; This lets you write something like (if (or COND1 COND2 COND3) (do-something) (do-something-else))      
  29     ;; Defines a macro that can manually structure the source code it is replaced with.                       
  30     ;; Allows for advanced compile time behavior.                                                             
  31     (defmacro or ARGS                                                                                         
  32         (if ARGS                                                                                              
  33             (qq                                                                                               
  34                 (if (unquote (f ARGS))                                                                        
  35                     1                              

In [10]:
clsp = Path('modules.clib').read_text()
print_clsp(clsp)

   1 (                                                                                                             
   2     (defun cons_42-fun (items)                                                                                
   3         (c "*" items)                                                                                         
   4     )                                                                                                         
   5                                                                                                               
   6     (defmacro cons_42-macro (items)                                                                           
   7         (qq                                                                                                   
   8             (c "*" (unquote items))                                                                           
   9         )                                                                                                     
  10     )                                                                                                         
  11                                                                                                               
  12     (defun add-fun (A B)                                                                                      
  13         (+ A B)                                                                                               
  14     )                                                                                                         
  15                                                                                                               
  16     (defun-inline add-inline (A B)                                                                            
  17         (+ A B)                                                                                               
  18     )                                                                                                         
  19                                                                                                               
  20     (defmacro add-macro (A B)                                                                                 
  21         (qq                                                                                                   
  22             (+ (unquote A) (unquote B))                                                                       
  23         )                                                                                                     
  24     )                                                                                                         
  25                                                                                                               
  26     (defun sum-fun (items)                                                                                    
  27         (if (r items)                                                                                         
  28             (+ (f items) (sum-fun (r items)))                                                                 
  29             (f items)                                                                                         
  30         )                                                                                                     
  31     )                                                                                                         
  32                                                                                                               
  33     (defmacro sum-macro items                                                                                 
  34         (if (r items)                                                                                         
  35             (list                                  

In [19]:
clsp = """
    (mod items
        (include modules.clib)
        (cons_42-fun items)
    )
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2 (mod items                                                                                                     
  3     (include modules.clib)                                                                                     
  4     (cons_42-fun items)                                                                                        
  5 )                                                                                                              
  6                                                                                                                

(a (q 2 2 (c 2 (c 3 ()))) (c (q 4 (q . 42) 5) 1))                                                                  

cost: 730

(42 1 2 3 4)                                                                                                       

In [20]:
clsp = """
    (mod items
        (include modules.clib)
        (cons_42-macro items)
    )
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2     (mod items                                                                                                 
  3         (include modules.clib)                                                                                 
  4         (cons_42-macro items)                                                                                  
  5     )                                                                                                          
  6                                                                                                                

(c (q . 42) 1)                                                                                                     

cost: 115

(42 1 2 3 4)                                                                                                       

In [2]:
clsp = """
  (mod (a b c d)                                                                                                 
      (include modules.clib)                                                                                     
      (sum-fun (list a b c d))                                                                                   
  )    
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
print_program(program.at("rrf"))
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2   (mod (a b c d)                                                                                               
  3       (include modules.clib)                                                                                   
  4       (sum-fun (list a b c d))                                                                                 
  5   )                                                                                                            
  6                                                                                                                

(a (q 2 2 (c 2 (c (c 5 (c 11 (c 23 (c 47 ())))) ()))) (c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1)) 

(c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1)                                                        

cost: 5758

10                                                                                                                 

In [1]:
clsp = """
  (mod (a b c d)                                                                                                 
      (include modules.clib)                                                                                     
      (sum-fun (list a b c))                                                                                   
  )    
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
print_program(program.at("rrf"))
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2   (mod (a b c d)                                                                                               
  3       (include modules.clib)                                                                                   
  4       (sum-fun (list a b c))                                                                                   
  5   )                                                                                                            
  6                                                                                                                

(a (q 2 2 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) (c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1))        

(c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1)                                                        

cost: 4177

6                                                                                                                  

In [15]:
clsp = """
 (mod (a b c d)                                                                                                 
     (include modules.clib)                                                                                     
     (sum-macro a b c d)                                                                                        
 )  
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2  (mod (a b c d)                                                                                                
  3      (include modules.clib)                                                                                    
  4      (sum-macro a b c d)                                                                                       
  5  )                                                                                                             
  6                                                                                                                

(+ 2 (+ 5 (+ 11 23)))                                                                                              

cost: 2484

10                                                                                                                 

In [16]:
clsp = """
 (mod (a b c)                                                                                                 
     (include modules.clib)                                                                                     
     (sum-macro a b c)                                                                                        
 )  
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
cost, result = program.run_with_cost(INFINITE_COST, Program.to([1, 2, 3, 4]))
print(f'cost: {cost}')
print_program(result)

  1                                                                                                                
  2  (mod (a b c)                                                                                                  
  3      (include modules.clib)                                                                                    
  4      (sum-macro a b c)                                                                                         
  5  )                                                                                                             
  6                                                                                                                

(+ 2 (+ 5 11))                                                                                                     

cost: 1668

6                                                                                                                  

# [Lambda](https://chialisp.com/examples#lambda)
> Compiles a block of code into a single executable expression. Useful for writing functions as arguments to other functions.

```clojure
(lambda (n1 n2)
    ;; Returns the two added together.
    (+ n1 n2)
)
```

In [21]:
clsp = """
(mod (v1 v2)
    (list
        (a 
            (lambda (n1 n2) (+ n1 n2)) ; use lambda
            (list v1 v2)
        )
        (a
            (q . (+ 2 5))              ; use quote expr
            (list v1 v2)
        )
    )
)
"""
print_clsp(clsp)
program = Program(compile_clvm_text(clsp, search_paths=["."]))
print_program(program)
cost, result = program.run_with_cost(INFINITE_COST, Program.to([100, 400]))
print(f'cost: {cost}')
print_program(result)

   1                                                                                                               
   2 (mod (v1 v2)                                                                                                  
   3     (list                                                                                                     
   4         (a                                                                                                    
   5             (lambda (n1 n2) (+ n1 n2)) ; use lambda                                                           
   6             (list v1 v2)                                                                                      
   7         )                                                                                                     
   8         (a                                                                                                    
   9             (q . (+ 2 5))              ; use quote expr                                                       
  10             (list v1 v2)                                                                                      
  11         )                                                                                                     
  12     )                                                                                                         
  13 )                                                                                                             
  14                                                                                                               

(c (+ 2 5) (c (+ 2 5) ()))                                                                                         

cost: 1884

(500 500)                                                                                                          